# Practical: Atomate 2

This is a practical session on using atomate 2 to run some calculations using the MACE potential that we used last time.


In [ ]:
!pip install atomate2

In [ ]:
from atomate2.ase.jobs import AseRelaxer
from pymatgen.core import Structure, Lattice
from mace.calculators.foundations_models import mace_mp

# Define the structure for silicon
lattice_si = Lattice.from_parameters(a=5.43, b=5.43, c=5.43, alpha=90, beta=90, gamma=90) 

silicon_structure = Structure.from_spacegroup(sg=227, species=["Si"], 
                                              lattice=lattice_si, 
                                              coords=[[0, 0, 0]])

# Create a 2x2x2 supercell
silicon_structure.make_supercell([2, 2, 2])
# Apply some rattling to the structure
silicon_structure.perturb(distance=0.1)

relaxer = AseRelaxer(calculator = mace_mp(model='medium',
                                        default_dtype='float64',
                                        device='cpu'),
                    relax_cell = True,
                    optimizer='LBFGS')

results = relaxer.relax(atoms = silicon_structure,
                        fmax = 0.01,
                        traj_file='silicon_relax.traj',
                        verbose=True)

Using Materials Project MACE for MACECalculator with /Users/zeyudeng/.cache/mace/20231203mace128L1_epoch199model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/Users/zeyudeng/apps/matsci/lib/python3.12/site-packages/mace/calculators/mace.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=de

       Step     Time          Energy          fmax
LBFGS:    0 04:24:08     -341.829770        0.136334
LBFGS:    1 04:24:08     -341.830564        0.135427
LBFGS:    2 04:24:09     -341.884174        0.043765
LBFGS:    3 04:24:09     -341.890577        0.001050


In [ ]:
print(f'Elapsed time: {results.elapsed_time:.2f} s')

for i,image in enumerate(results.trajectory):
    print(f'Step{i}: Lattice constant: {image.lattice.abc} Angstrom')


Elapsed time: 2.12 s
Step0: Lattice constant: (10.86, 10.86, 10.86) Angstrom
Step1: Lattice constant: (10.860330493256305, 10.860330493256305, 10.860330493256305) Angstrom
Step2: Lattice constant: (10.894322110283861, 10.89432211028484, 10.89432211028689) Angstrom
Step3: Lattice constant: (10.910589533719515, 10.910589533720819, 10.910589533723915) Angstrom
Step4: Lattice constant: (10.910589533719515, 10.910589533720819, 10.910589533723915) Angstrom


In [ ]:
from atomate2.ase.jobs import AseRelaxMaker
from jobflow import run_locally

atoms = silicon_structure.to_ase_atoms()
atoms.calc = mace_mp(model='medium',
                     default_dtype='float64',
                     device='cpu')

workflow = AseRelaxMaker(name='silicon_relax',
                         relax_cell=True,
                         optimizer='LBFGS',
                         fmax=0.01,
                         ionic_step_data = ['energy', 'forces', 'stress'],
                         traj_file='silicon_relax.traj',
                         trajectory_interval=1,
                         trajectory_verbose=True).make(atoms)

run_locally(workflow, atoms)
# print(f'Elapsed time: {workflow.results.elapsed_time:.2f} s')


Using Materials Project MACE for MACECalculator with /Users/zeyudeng/.cache/mace/20231203mace128L1_epoch199model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/Users/zeyudeng/apps/matsci/lib/python3.12/site-packages/mace/calculators/mace.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=de

TypeError: Can't instantiate abstract class AseRelaxMaker without an implementation for abstract method 'calculator'